# **Réalisez un traitement dans un environnement Big Data sur le Cloud**

## **Import des librairies**

In [1]:
import time
import io
import os

import pandas as pd
from PIL import Image
import numpy as np

# Désactivation des messages de debugging de tensorflow.
# Doit être exécuté avant les imports de tensorflow.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

### **Mesure de la durée d'exécution du notebook**

In [2]:
start_time = time.perf_counter()

## **Définition des PATH pour charger les images et enregistrer les résultats**

In [3]:
PATH = os.getcwd()
PATH_Data = PATH + '/data/Test1'
PATH_Result = PATH + '/data/Results'
print('PATH :        ' + PATH\
      + '\nPATH_Data :   ' + PATH_Data \
      + '\nPATH_Result : ' + PATH_Result)

PATH :        /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers
PATH_Data :   /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Test1
PATH_Result : /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Results


## **Création de la SparkSession**

In [4]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Nous créons également la variable "**sc**" qui est un **SparkContext** issue de la variable **spark** :

In [5]:
sc = spark.sparkContext

Affichage des informations de Spark en cours d'execution :

In [6]:
spark

## **Traitement des données**

### **Chargement des données**

In [7]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

Je ne conserve que le **path** de l'image et j'ajoute une colonne contenant les **labels** de chaque image :

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,True)) # False si on ne veut pas tronquer les colonnes lors de l'affichage

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------+------------------+
|                path|             label|
+--------------------+------------------+
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
+--------------------+------------------+
only showing top 5 rows

None


### **Préparation du modèle**

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** <br />
    issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [11]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

<u>Mettons cela sous forme de fonction</u> :

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

### **Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF**

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/home/maxime/.local/lib/python3.10/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


### **Exécution des actions d'extraction de features**

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), <br />
peuvent rencontrer des erreurs de type Out Of Memory (OOM).<br />
Si vous rencontrez de telles erreurs dans la cellule ci-dessous, <br />
essayez de réduire la taille du lot Arrow via 'maxRecordsPerBatch'

Je n'utiliserai pas cette commande dans ce projet <br />
et je laisse donc la commande en commentaire.

In [15]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [16]:
# Choix du nombre de partitions que l'on va créer avec "images"
nb_partitions = 20

features_df = images.repartition(nb_partitions).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [17]:
features_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)



### **Réalisation de la PCA**

Conversion de la colonne features en vecteurs SparseVector, car c'est le format d'entrée requis pour faire une PCA

In [18]:
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
features_df = features_df.withColumn('features', list_to_vector_udf('features'))

On choisit le nombre de composantes à garder avec la PCA : 

In [19]:
n_componants = 20

In [20]:
pca = PCA(k=n_componants, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(features_df)
features_df = model.transform(features_df).select("path", "label", "features", "pcaFeatures")

1/1 [==============================] - 3s 3s/step                   (0 + 1) / 1]
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
1/1 [==============================] - 0s 491ms/step===========>  (19 + 1) / 20]


In [21]:
print(f"Ensemble, ces {n_componants} composantes captent {sum(model.explainedVariance)*100:.2f} % de la variance.")

Ensemble, ces 20 composantes captent 91.78 % de la variance.


In [22]:
features_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- pcaFeatures: vector (nullable = true)



### **Écriture des résultats sous forme de fichiers**

In [23]:
print(f"Les résultats iront ici :\n{PATH_Result}")

Les résultats iront ici :
/mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Results


Enregistrement des données traitées au format "**parquet**" :

In [24]:
features_df.write.mode("overwrite").parquet(PATH_Result)

1/1 [==============================] - 1s 510ms/step===========>  (19 + 1) / 20]


## **Chargement des données enregistrées et validation du résultat**

On charge les données qui ont été enregistrées au format parquet dans un **DataFrame Pandas** :

In [25]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

On ne va garder que les noms de fichiers au lieu de tout le chemin : 

In [26]:
# Séparer les éléments de 'path' et ne garder que le dernier (nom du fichier)
df['path'] = df['path'].apply(lambda x: x.split('/')[-1])
# Renommer la colonne 'path' en 'filename'
df = df.rename(columns={'path': 'filename'})

Ici les valeurs prennent la forme d'un dictionnaire, car *features* avait été converti en vecteurs spark

In [27]:
df['features'][0]

{'type': 1,
 'size': None,
 'indices': None,
 'values': array([0.        , 0.        , 0.01411368, ..., 0.        , 0.00696147,
        0.        ])}

Nous souhaitons seulement avoir les valeurs : 

In [28]:
df['features'] = df['features'].apply(lambda x: x['values'] if x is not None else None)
df['pcaFeatures'] = df['pcaFeatures'].apply(lambda x: x['values'] if x is not None else None)

On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280 :

In [29]:
df.loc[0,'features'].shape

(1280,)

On valide dimension du vecteur de caractéristiques après PCA, qui doit correspondre au nombre choisi de composantes lors de la PCA : 

In [30]:
df.loc[0,'pcaFeatures'].shape

(20,)

In [31]:
df.head()

,filename,label,features,pcaFeatures
0,r_22_100.jpg,Apple Crimson Snow,"[0.0, 0.0, 0.01411367580294609, 0.0, 0.0, 0.03...","[-1.0326404993020393, -4.81824508227784, 0.527..."
1,r_41_100.jpg,Apple Crimson Snow,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.2474969625473022, ...","[3.746016535554548, -4.938483473127772, -1.597..."
2,r_35_100.jpg,Apple Braeburn,"[0.6390393972396851, 0.03641435131430626, 0.0,...","[-12.987896923687625, -5.113362345556348, 1.61..."
3,r_42_100.jpg,Apple Braeburn,"[0.8058856129646301, 0.07883457094430923, 0.0,...","[-14.859889576493904, -6.183629719520848, 3.37..."
4,r_113_100.jpg,Apple Crimson Snow,"[0.015286029316484928, 0.07301358133554459, 0....","[-11.104970357570675, -1.1929954589661127, -0...."


Enregistrement des pcaFeatures (associées aux noms de fichier et labels) au format CSV : 

In [32]:
# np.printoptions pour éviter l'insertion de "\n" dans 'pcaFeatures' dans notre fichier csv
with np.printoptions(linewidth=10000):
    df[['filename', 'label', 'pcaFeatures']].to_csv(PATH_Result+'/'+'pcaFeatures.csv', index=False, sep='\t')

### **Mesure de la durée d'exécution du notebook**

In [33]:
end_time = time.perf_counter()
elapsed_time = end_time - start_time

# Conversion en minutes et secondes
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"Durée d'exécution du notebook (hors imports) : {minutes:02}:{seconds:02}")

Durée d'exécution du notebook (hors imports) : 03:06
